In [40]:
import pandas as pd
import numpy as np
import plotly.express as px
import chardet
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from datetime import datetime


# ファイルのエンコーディングを自動検出
with open('data.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read())

In [41]:
sheet_list = ["2010年", "2011年", "2012年", "2013年", "2014年", "2015年", "2016年",
              "2017年", "2018年", "2019年", "2020年", "2021年", "2022年"]
df_dict = pd.read_excel("売上 - 分析.xlsx", sheet_name=sheet_list)

In [42]:
df = pd.concat(df_dict.values())

In [43]:
temperature_df = pd.read_csv("data.csv", encoding='shift-jis')

In [44]:
df["品目"].value_counts().index

Index(['小松菜                              ', 'ピーマン                            ',
       'ズッキーニ                          ', '小松菜',
       'きゅうり                            ', 'みず菜                              ',
       'かぶ                                ', 'ズッキーニ',
       'サニーレタス                        ', 'サニーレタス',
       ...
       '送料(クール便)岡山県', '送料長野県', '葉たまねぎ', 'ひもとう', '玉ねぎ　                     ',
       '葉付にんじん                        ', '送料(宮城-常温)', '送料-北海道',
       '送料 ゆうパックチルド　奄美市', '送料　奄美市（ゆうパックチルド）'],
      dtype='object', name='品目', length=308)

In [45]:
df_cucumber = df[df["品目"].str.contains("きゅうり")]
df_cucumber["品目"].value_counts().index

Index(['きゅうり                            ', 'きゅうり',
       'ミニきゅうり                        ', 'レモンきゅうり                      ',
       'きゅうり規格外品'],
      dtype='object', name='品目')

In [46]:
df_cucumber["date"] = pd.to_datetime({
    "year": df_cucumber['年'],
    "month": df_cucumber['月'],
    "day": df_cucumber['日']
})

/var/folders/km/rlhk3k8x73v_63b8j0frtmlr0000gn/T/ipykernel_84804/1383750078.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [47]:
df_cucumber.set_index("date", inplace=True)

In [48]:
df_cucumber = df_cucumber[["個数"]]

In [49]:
df_cucumber = df_cucumber.groupby("date").sum()

In [50]:
temperature_df["date"] = pd.to_datetime(temperature_df["年月日"])
temperature_df.set_index("date", inplace=True)
temperature_df = temperature_df[["平均気温(℃)"]]

In [51]:
result_df = pd.merge(temperature_df, df_cucumber,
                     left_index=True, right_index=True, how="outer")
result_df = result_df.resample("3D").sum()
result_df["平均気温(℃)"] = result_df["平均気温(℃)"]/3

In [52]:
def create_year_df(year):
    start_date = datetime(year, 1, 1)
    end_date = datetime(year, 12, 31)
    return result_df[(result_df.index >= start_date) & (result_df.index <= end_date)]

In [53]:
# サブプロットを作成
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 第1のy軸に対応するグラフを追加
fig.add_trace(
    go.Scatter(x=result_df.index, y=result_df["個数"], name="個数"),
    secondary_y=False,
)

# 第2のy軸に対応するグラフを追加
fig.add_trace(
    go.Scatter(x=result_df.index, y=result_df["平均気温(℃)"], name="温度"),
    secondary_y=True,
)

# レイアウトを設定
fig.update_layout(
    title="2010-2022 きゅうり\n 相関係数" +
    str(result_df["個数"].corr(result_df["平均気温(℃)"])),
    xaxis=dict(title="日付"),
    yaxis=dict(title="個数", side="left"),
    yaxis2=dict(title="温度", side="right"),
)

# グラフを表示
fig.show()
pio.write_html(fig, f'figure_html/cucumber.html')

In [54]:
result_df["平均気温(℃)"]

date
2003-01-01   -2.066667
2003-01-04   -3.100000
2003-01-07   -3.066667
2003-01-10    0.433333
2003-01-13   -0.500000
                ...   
2022-12-17   -0.666667
2022-12-20    0.700000
2022-12-23   -0.833333
2022-12-26    0.466667
2022-12-29    1.833333
Name: 平均気温(℃), Length: 2435, dtype: float64

In [55]:
for year in range(2010, 2023):
    # サブプロットを作成
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # 第1のy軸に対応するグラフを追加
    fig.add_trace(
        go.Scatter(x=create_year_df(year).index,
                   y=create_year_df(year)["個数"], name="個数"),
        secondary_y=False,
    )

    # 第2のy軸に対応するグラフを追加
    fig.add_trace(
        go.Scatter(x=create_year_df(year).index,
                   y=create_year_df(year)["平均気温(℃)"], name="温度"),
        secondary_y=True,
    )

    # レイアウトを設定
    fig.update_layout(
        title=f"{year} きゅうり\n 相関係数 " +
        str(create_year_df(year)["個数"].corr(create_year_df(year)["平均気温(℃)"])),
        xaxis=dict(title="日付"),
        yaxis=dict(title="個数", side="left"),
        yaxis2=dict(title="温度", side="right"),
    )
    pio.write_image(fig, f'figure/cucumber_plot_{year}.svg')
    pio.write_html(fig, f'figure_html/cucumber_plot_{year}.html')

In [56]:
for year in range(2010, 2023):
    df1 = create_year_df(year).copy()
    for n in range(1, 11):
        df1[f"{n*3}days ago"] = create_year_df(year)["個数"].shift(n)
    correlation = df1.corr()["平均気温(℃)"].iloc[1:]
    max_index = correlation.abs().idxmax()

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
        go.Scatter(x=df1.index,
                   y=df1[max_index], name="個数"),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df1.index,
                   y=df1["平均気温(℃)"], name="温度"),
        secondary_y=True,
    )
    fig.update_layout(
        title=f"{year} きゅうり, 相関係数{max_index}",
        xaxis=dict(title="日付"),
        yaxis=dict(title=max_index, side="left"),
        yaxis2=dict(title="温度", side="right"),
    )
    pio.write_image(fig, f'figure/cucumber_plot_{year}_max_correlation.svg')
    # pio.write_html(
    #     fig, f'figure_html/cucumber_plot_{year}_max_correlation.html')
    print(year, max_index, correlation[max_index])

2010 個数 0.7541512084908619
2011 個数 0.6108641036871681
2012 個数 0.6537360895052463
2013 6days ago 0.6949082010229324
2014 個数 0.6303644039177384
2015 9days ago 0.6691860564283135
2016 個数 0.6179559489951084
2017 個数 0.6474472017831951
2018 個数 0.6387625138387547
2019 個数 0.6391211489680649
2020 個数 0.699117356940471
2021 個数 0.6606167228263199
2022 個数 0.6666972776137161


In [57]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
train_df = result_df
# train_df = result_df[result_df.index]
# データを訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(
    train_df["平均気温(℃)"], train_df["個数"], test_size=0.2, random_state=42)

# XGBoostの回帰モデルを訓練
model = xgb.XGBRegressor(objective='reg:squarederror')
model.fit(X_train, y_train)

# テストデータで予測
y_pred = model.predict(X_test)

# モデルの評価（例：平均二乗誤差）
mse = mean_squared_error(y_test, y_pred)
print(f"平均二乗誤差: {mse}")

平均二乗誤差: 19822.9202181702


In [58]:
for year in range(2010, 2023):
    create_year_df(year).to_csv(f"data/{year}_df")